In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
lab_test = pd.read_csv('/kaggle/input/cleaned_full_No_CVD_EHR_medical.csv', delimiter=',')
lab_test['age'] = lab_test['AGE_END2017']
lab_test.drop('AGE_END2017',inplace=True,axis=1)


In [ ]:
import time

def find_notnull(data):
    """ Finds the indices of all missing values.
    Parameters
    ----------
    data: numpy.ndarray
        Data to impute.
    Returns
    -------
    List of tuples
        Indices of all missing values in tuple format; (i, j)
    """
    null_xy = np.argwhere(~np.isnan(data))
    return null_xy

def EMA_moving_window(visit_idx_in_window, col_feature, data_within_moving_window, default_value, time_col='ENCOUNTER_DT_RAN', hlife=30):
    timeseries = np.array(data_within_moving_window[time_col])
    pivot = timeseries[visit_idx_in_window]
    time_difference = np.abs(timeseries - pivot) / hlife
    
    notnan_data_indices = find_notnull(data_within_moving_window[col_feature])
    if len(notnan_data_indices) == 0:
        return default_value
    
    exp_weights = np.power(0.5, time_difference[notnan_data_indices])
    return sum(np.array(data_within_moving_window[col_feature])[notnan_data_indices] * exp_weights) / sum(exp_weights)

# def EMA_moving_window(visit_idx_in_window, col_feature, data_within_moving_window, default_value, time_col='ENCOUNTER_DT_RAN', hlife=30):
#     timeseries = np.array(data_within_moving_window[time_col])
#     pivot = timeseries[visit_idx_in_window]
#     time_difference = np.abs(timeseries - pivot) / hlife
    
#     notnan_data_indices = find_notnull(data_within_moving_window[col_feature])
#     if len(notnan_data_indices) == 0:
#         return default_value
    
#     exp_weights = np.power(0.5, time_difference[notnan_data_indices])
#     return np.array(data_within_moving_window[col_feature])[notnan_data_indices] * exp_weights / sum(exp_weights)


demo_feature_list = ['study_id','ENCOUNTER_DT_RAN','age','smoke','SEX','RACE']
interest_feature_list = [
 'CREATININE',
 'HEMOGLOBIN A1C',
 'CHOLESTEROL, TOTAL',
 'HDL CHOLESTEROL',
 'TRIGLYCERIDES',
 'EGFR MDRD AFRICAN AMERICAN',
 'LDL CHOLESTEROL',
 'EGFR MDRD NON AFRICAN AMERICAN',
 'CREATININE, URINE',
 'BP_SYSTOLIC','BP_DIASTOLIC',
 'BMI']


data_lab_tests = []
cnt= 0
#timestamp = time.time()
for patient, d in tqdm(lab_test.groupby(['study_id'])):
#     if cnt % 100 == 0: 
#         print(cnt)
#         print(time.time() - timestamp)
#         timestamp = time.time()
    cnt += 1
    cur_half_year_idx = 0
    total_num_visits = d.shape[0]
    filled_data_d = np.zeros((d.shape[0],len(interest_feature_list)))
    for visit_idx in range(d.shape[0]):
        # EMA impute
        while d.iloc[visit_idx,0] - d.iloc[cur_half_year_idx,0] >= 61: 
            cur_half_year_idx += 1
        
        #temp_past_half_year = d.iloc[cur_half_year_idx:visit_idx+1]
        next_half_year_idx = visit_idx
        while next_half_year_idx < total_num_visits and d.iloc[next_half_year_idx,0] - d.iloc[visit_idx,0] <= 30: # quater
            next_half_year_idx += 1
        
        visit_idx_in_window = visit_idx - cur_half_year_idx
        data_within_moving_window = d.iloc[cur_half_year_idx:min(next_half_year_idx+1,d.shape[0])]
        #cur_visit_lab = d.iloc[visit_idx] #temp_past_half_year.iloc[-1]
        
        for col_cnt, lab_test_ftr in enumerate(interest_feature_list): 
            if np.isnan(d[lab_test_ftr].iloc[visit_idx]):
                #data_within_moving_window.apply(lambda x: EMA_moving_window(visit_idx_in_window, lab_test_ftr, np.nanmean(d[lab_test_ftr])), axis=0)
                d[lab_test_ftr].iloc[visit_idx] = EMA_moving_window(visit_idx_in_window, lab_test_ftr, data_within_moving_window, np.nanmean(d[lab_test_ftr]))
        
    #d[interest_feature_list] = filled_data_d
    data_lab_tests.append(d)

# data_lab_tests
# filled_lab_test = pd.DataFrame(data_lab_tests)#, columns=demo_feature_list + interest_feature_list)

In [ ]:
# data_lab_tests
# filled_lab_test = pd.DataFrame(data_lab_tests,columns=lab_test.columns)
# filled_lab_test
filled_lab_test = pd.concat(data_lab_tests)
filled_lab_test.to_csv('cleaned_full_No_CVD_EHR_medical_forward_backward_EMA_filling.csv',index=False)